# Instrument Classification - Part 3 : Track Prediction

In [ ]:
import numpy as np
import librosa
import os
import keras

In [ ]:
def crop_song(dir,songs):
    predict = [] #this must be array of lists for songs>1
    even = True
    it = 0
    for f in os.listdir(dir):
        if (it == songs):
            break
        if (even): 
            even = False 
            continue
        else:
            even = True
            wav,sr = librosa.load(dir+f,mono=True)
            for i in range(0,int(len(wav)/sr)):
                crop_wav = wav[i*sr:(i+1)*sr]
                crop_wav = crop_wav / np.sqrt(np.mean(crop_wav**2))
                mel = librosa.feature.melspectrogram(crop_wav,n_mels=96,n_fft=1024,hop_length=256)
                predict.append(mel)
            crop_wav = wav[(i+1)*sr:]
            crop_wav = crop_wav / np.sqrt(np.mean(crop_wav**2))
            mel = librosa.feature.melspectrogram(crop_wav,n_mels=96,n_fft=1024,hop_length=256)
            predict.append(mel)
        it = it+1
    return predict

def extract_instruments_from_prediction(prediction):
    print()

In [ ]:
dir = './Desktop/Instrument-Classification/IRMAS-TestingData-Part1/Part1/'

songs = 1
predict = crop_song(dir,songs)

predict_new = np.expand_dims(np.array(predict[:-1]),-1) 
#last one doesn't have the same length so we can't make an array
#easily solvable by padding the last one with zeros

In [ ]:
model = keras.models.load_model('ICmodel.h5')
instr = []
for i in range(0,len(predict_new)):
    instr.append([])
    prediction = model.predict(predict_new[i:i+1])
    instr[i].append(extract_instruments_from_prediction(prediction))

#printed example:
# 0-1s : guitar,piano
# 1-2s : guitar,piano
# 2-3s : guitar
# 3-4s : guitar,voice
# 4-5s : voice
# 5-end: piano